In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pca-data/large_pca_dataset.csv


In [2]:
df=pd.read_csv("/kaggle/input/pca-data/large_pca_dataset.csv")

In [3]:
df.head()

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20
0,0.496714,-0.138264,0.647689,1.523030,-0.234153,-0.234137,1.579213,0.767435,-0.469474,0.542560,-0.463418,-0.465730,0.241962,-1.913280,-1.724918,-0.562288,-1.012831,0.314247,-0.908024,-1.412304
1,1.465649,-0.225776,0.067528,-1.424748,-0.544383,0.110923,-1.150994,0.375698,-0.600639,-0.291694,-0.601707,1.852278,-0.013497,-1.057711,0.822545,-1.220844,0.208864,-1.959670,-1.328186,0.196861
2,0.738467,0.171368,-0.115648,-0.301104,-1.478522,-0.719844,-0.460639,1.057122,0.343618,-1.763040,0.324084,-0.385082,-0.676922,0.611676,1.031000,0.931280,-0.839218,-0.309212,0.331263,0.975545
3,-0.479174,-0.185659,-1.106335,-1.196207,0.812526,1.356240,-0.072010,1.003533,0.361636,-0.645120,0.361396,1.538037,-0.035826,1.564644,-2.619745,0.821903,0.087047,-0.299007,0.091761,-1.987569
4,-0.219672,0.357113,1.477894,-0.518270,-0.808494,-0.501757,0.915402,0.328751,-0.529760,0.513267,0.097078,0.968645,-0.702053,-0.327662,-0.392108,-1.463515,0.296120,0.261055,0.005113,-0.234587


In [4]:
import matplotlib.pyplot as plt

In [5]:
X=df.iloc[:,1:]
Y=df.iloc[:,0]

In [8]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [10]:
X_train.shape

(4000, 19)

In [14]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor(n_neighbors=5)
knn.fit(X_train, Y_train)


KNeighborsRegressor()

In [17]:
import time
start =time.time()
Y_pred = knn.predict(X_test)
print(time.time()-start)

0.020275115966796875


In [20]:
# 1. Convert Y to binary class
threshold = Y_train.mean()

Y_train_class = (Y_train > threshold).astype(int)
Y_test_class  = (Y_test  > threshold).astype(int)

# 2. Train classifier (NOT regressor)
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train, Y_train_class)   # <-- IMPORTANT!

# 3. Predict
Y_pred = knn.predict(X_test)

# 4. Accuracy
from sklearn.metrics import accuracy_score
print(accuracy_score(Y_test_class, Y_pred))


0.489


In [21]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [22]:
X_train= scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [23]:
from sklearn.decomposition import PCA

pca=PCA(n_components=None)

In [28]:
X_train_trf=pca.fit_transform(X_train)
X_test_trf=pca.transform(X_test)

In [29]:
X_train_trf.shape

(4000, 19)

In [31]:

knn = KNeighborsClassifier(n_neighbors=5)

In [35]:
# 1. Convert continuous Y into binary classes
threshold = Y_train.mean()

Y_train_class = (Y_train > threshold).astype(int)
Y_test_class  = (Y_test  > threshold).astype(int)

# 2. Train classifier
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

knn.fit(X_train_trf, Y_train_class)   # <-- IMPORTANT

# 3. Predict
Y_pred = knn.predict(X_test_trf)

# 4. Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy =", accuracy_score(Y_test_class, Y_pred))


Accuracy = 0.493


In [37]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

# 1. Convert Y to classes for classification
threshold = Y_train.mean()
Y_train_class = (Y_train > threshold).astype(int)
Y_test_class  = (Y_test  > threshold).astype(int)

# 2. Loop for PCA
for i in range(1, 21):

    pca = PCA(n_components=i)
    X_train_trf = pca.fit_transform(X_train)
    X_test_trf  = pca.transform(X_test)

    # 3. Classifier MUST use Y_train_class (not Y_train)
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train_trf, Y_train_class)

    Y_pred = knn.predict(X_test_trf)

    print(f"PCA Components = {i},  Accuracy = {accuracy_score(Y_test_class, Y_pred)}")


PCA Components = 1,  Accuracy = 0.52
PCA Components = 2,  Accuracy = 0.484
PCA Components = 3,  Accuracy = 0.506
PCA Components = 4,  Accuracy = 0.506
PCA Components = 5,  Accuracy = 0.534
PCA Components = 6,  Accuracy = 0.518
PCA Components = 7,  Accuracy = 0.497
PCA Components = 8,  Accuracy = 0.501
PCA Components = 9,  Accuracy = 0.505
PCA Components = 10,  Accuracy = 0.501
PCA Components = 11,  Accuracy = 0.519
PCA Components = 12,  Accuracy = 0.494
PCA Components = 13,  Accuracy = 0.489
PCA Components = 14,  Accuracy = 0.483
PCA Components = 15,  Accuracy = 0.477
PCA Components = 16,  Accuracy = 0.492
PCA Components = 17,  Accuracy = 0.47
PCA Components = 18,  Accuracy = 0.502
PCA Components = 19,  Accuracy = 0.493


ValueError: n_components=20 must be between 0 and min(n_samples, n_features)=19 with svd_solver='full'

In [38]:
pca=PCA(n_components=2)
X_train_trf=pca.fit_transform(X_train)
X_test_trf=pca.transform(X_test)

In [39]:
X_train_trf

array([[ 1.8173545 , -0.82066012],
       [ 0.10943622, -1.79629513],
       [-0.82176961,  0.35774479],
       ...,
       [ 0.37335197, -1.10712041],
       [ 0.53799009, -0.72997527],
       [-1.45496861, -0.77208483]])

In [43]:
fig = px.scatter(
    x=X_train_trf[:, 0],
    y=X_train_trf[:, 1],
    color=Y_train_trf,
    color_discrete_sequence=px.colors.qualitative.Set1
)

fig.show()


In [44]:
pca=PCA(n_components=3)
X_train_trf=pca.fit_transform(X_train)
X_test_trf=pca.transform(X_test)

In [45]:
X_train_trf

array([[-1.31436633,  0.10852366,  1.44533882],
       [ 0.39476692, -0.21725628,  0.42762512],
       [ 0.7422169 , -0.031464  , -1.39853043],
       ...,
       [ 0.17154131,  0.03038423,  2.47385286],
       [-0.8665181 , -2.57354441, -0.17070319],
       [ 0.67057671,  0.22854593, -0.94965026]])

In [47]:
import plotly.express as px

Y_train_trf = Y_train.astype(str)

fig = px.scatter_3d(
    x=X_train_trf[:, 0],
    y=X_train_trf[:, 1],
    z=X_train_trf[:, 2],
    color=Y_train_trf
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20)
)

fig.show()


In [48]:
pca.explained_variance_

array([1.1145814 , 1.09090817, 1.05250674])

In [49]:
pca.components_

array([[-0.96024645,  0.13643294,  0.17103807,  0.04024372, -0.02993252,
        -0.00447788,  0.05803524, -0.0032839 ,  0.09601028,  0.01976728,
        -0.07299812, -0.05038085,  0.01433828,  0.00349051,  0.02947743,
        -0.01614748,  0.00841088, -0.05496028,  0.04722147],
       [ 0.11984525,  0.96306698,  0.06244856, -0.10223958,  0.14118994,
         0.00950119, -0.01252723,  0.04163928, -0.03010573, -0.04955556,
         0.08926116,  0.01981892, -0.01854105, -0.01832139, -0.02539628,
        -0.03519688, -0.0019039 ,  0.08264239, -0.02741612],
       [ 0.04056285, -0.09110447,  0.53646325,  0.21993078,  0.63719249,
        -0.09317859, -0.07163682, -0.28886288, -0.31338118,  0.05892276,
        -0.00772158,  0.10843029,  0.00571378,  0.09786269,  0.00502448,
         0.09567072,  0.09390792, -0.09128381,  0.03444434]])

In [50]:
pca.explained_variance_ratio_

array([0.05864751, 0.05740187, 0.05538124])